In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml

In [ ]:
! pip install --quiet git+https://github.com/nandanrao/facebook-python-business-sdk@pagination

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env
env = Env()
env.read_env('.env-bug')

In [552]:
from malaria import get_conf, get_df, lookalike
from marketing import Marketing

cnf = get_conf(env)
df = get_df(cnf)
m = Marketing(env)

In [ ]:
from malaria import update_audience, update_ads

# update_audience()

update_ads()

In [150]:
from datetime import datetime, timezone
from clustering import only_target_users

def filter_time(df, min_date, max_date):
    min_date = datetime(*min_date, tzinfo=timezone.utc)
    max_date = datetime(*max_date, tzinfo=timezone.utc)

    users = df[(df.timestamp > min_date) & (df.timestamp < max_date)].userid.unique()
    return df[df.userid.isin(users)]

def get_percentage(df, cnf):
    answered = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(answered)]
    tot = df.userid.unique().shape[0]
    targets = only_target_users(df, cnf['strata'][0]['surveys'], 'target_questions')
    targets = targets.userid.unique().shape[0]
    return targets/tot, targets, tot


In [44]:
import pandas as pd
from clustering import add_res_cols
from malaria import load_cities


def perc_target(users, df):
    u = pd.Series(df.userid.unique())
    tot = u.unique().shape[0]
    target = u.isin(users).sum()
    perc = target / tot
    return pd.Series([perc, tot, target], index=['perc', 'tot', 'target'])


def make_analysis_df(df, districts, users):
    finished = df[df.question_ref == 'f6e69027-97cc-494e-8d52-318b75047e23'].userid.unique()
    df = df[df.userid.isin(finished)].reset_index(drop=True)
    df = add_res_cols('md:clusterid', 'cluster', 'userid', df)
    perc = df.groupby('cluster').apply(lambda df: perc_target(users, df)).reset_index()
    perc = perc.merge(districts, left_on='cluster', right_on='disthash')
    return perc

In [45]:

cities = load_cities('output/cities.csv')

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True) \
                  .drop(columns=['rad', 'lng', 'lat', 'distcode', 'id'])


# perc = make_analysis_df(df, districts, users)

In [ ]:
def statewide(df):
    trg = df.target.sum().astype(int)
    tot = df.tot.sum().astype(int)
    return pd.Series([trg/tot, trg, tot], index=['percentage', 'malaria', 'total'])

states = perc.groupby('state').apply(statewide).round(4)
states['total'] = states['total'].astype(int)
states['malaria'] = states['malaria'].astype(int)

states

In [122]:
chosen_clusters = perc[(perc.target >= 3) & (perc.perc > 0.1)].cluster
top_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)

In [ ]:
perc[perc.perc > .35].sort_values('target')

In [90]:
chosen_clusters = perc[(perc.perc > .35) & (perc.target >= 3)].cluster
malaria_cities = cities[cities.disthash.isin(chosen_clusters)].reset_index(drop=True)
malaria_cities = malaria_cities[~malaria_cities.state.isin(['Orissa'])]
malaria_cities.to_csv('output/malaria-cities.csv', index=False)

In [ ]:
malaria_cities

In [94]:
import pandas as pd

top_cities = pd.read_csv('output/top-cities.csv')

In [ ]:
top_cities

In [26]:

def pause_adsets(m, clusters):
    get_ad = lambda c: next((a for k, a in m.running_ads.items() if c in k), None)
    to_delete = [get_ad(c) for c in clusters]
    upd = lambda a: a.api_update(params={ 'status': 'PAUSED' })
    return [upd(ad) for ad in to_delete]
        
# res = pause_adsets(m, perc[perc.perc == 0.0].cluster)

In [ ]:
aud = get_aud(m, cnf, True)

In [ ]:
aud

In [9]:
from malaria import unsaturated

clusters = unsaturated(df, cnf)

In [75]:
adsets = m.account.get_ad_sets(fields=['lifetime_budget', 'instagram_actor_id', 'campaign_id', 'targeting'])

In [55]:
import hashlib
import hmac

def secret_proof(app_secret, access_token):
    h = hmac.new (
        app_secret.encode('utf-8'),
        msg=access_token.encode('utf-8'),
        digestmod=hashlib.sha256
    )
    return h.hexdigest()


secret = env('FACEBOOK_APP_SECRET')
token = ''
user = users[0]

proof = secret_proof(secret, token)

import requests

res = requests.get(f'https://graph.facebook.com/v7.0/{user}/ids_for_apps?access_token={token}&appsecret_proof={proof}')

res.text

'{"error":{"message":"(#100) An owning business is required for this request","type":"OAuthException","code":100,"fbtrace_id":"AypFAF9YX6tqhTqMedHqQHB"}}'

In [33]:
import time

from facebook_business.adobjects.serverside.event import Event
from facebook_business.adobjects.serverside.event_request import EventRequest
from facebook_business.adobjects.serverside.user_data import UserData
from facebook_business.adobjects.serverside.custom_data import CustomData
from facebook_business.api import FacebookAdsApi

# access_token = '<ACCESS_TOKEN>'
# pixel_id = 'ADS_PIXEL_ID>'


# user_data = UserData(
#     email='joe@eg.com',
#     fbc='fb.1.1554763741205.AbCdEfGhIjKlMnOpQrStUvWxYz1234567890',
#     fbp='fb.1.1558571054389.1098115397'
# )

# custom_data = CustomData(
#     currency='usd',
#     value=123.45
# )

# event = Event(
#     event_name='Purchase',
#     event_time=int(time.time()),
#     user_data=user_data,
#     custom_data=custom_data,
# )

# events = [event]

# event_request = EventRequest(
#     events=events,
#     pixel_id=pixel_id)

# event_response = event_request.execute()
# print(event_response)

In [6]:

cities = load_cities('output/cities.csv')



In [ ]:
from marketing import get_creatives, get_label
from facebook_business.adobjects.adcreative import AdCreative

def delete_our_creatives(account, label):
    label = get_label(account, label)
    creatives = get_creatives(account, label['id'])
    print(len(creatives))

    i = 0
    for c in creatives.values():
        try:
            c.api_delete()
            i += 1
            print(i)
        except Exception as e:
            print(e)

# delete_our_creatives(m.account, 'virtuallab-1')

In [16]:
from responses import get_forms
from datetime import datetime

survey_user = '10383123-9fb1-429b-8312-447c1b54b17a'
shortcodes = ['baselinehin', 'baselineeng', 'baselineodi']

timestamp = datetime(2020, 7, 25)

forms = list(get_forms(survey_user, shortcodes, timestamp, cnf['chatbase']))

In [432]:
forms[0]

{'id': 'Yym3bbfC',
 'title': 'baselineeng',
 'workspace': {'href': 'https://api.typeform.com/workspaces/p7LWDc'},
 'theme': {'href': 'https://api.typeform.com/themes/qHWOQ7'},
 'settings': {'language': 'en',
  'progress_bar': 'percentage',
  'meta': {'allow_indexing': False},
  'is_public': True,
  'is_trial': False,
  'show_progress_bar': True,
  'show_typeform_branding': True,
  'are_uploads_public': False},
 'thankyou_screens': [{'id': 'DefaultTyScreen',
   'ref': 'default_tys',
   'title': 'Done! Your information was sent perfectly.',
   'properties': {'show_button': False, 'share_icons': False}}],
 'fields': [{'id': '0urmFpT3hBxK',
   'title': 'Welcome {{hidden:name}}! 😀\nWe are a team of researchers conducting a study about life and opinions of Indian people. We would like to ask you a few questions today and only five questions every two weeks over the following six months.\n \nIf you complete the whole study, you will receive 300 INR in mobile credit. In particular, you will ge

In [575]:
questions = [
    ('malaria_incidence', 'f6e69027-97cc-494e-8d52-318b75047e23'),
    ('malaria_now', 'dad23031-8468-4900-89cc-d01841d8b660'),
    ('dist_medical', 'd7573919-8a7e-457f-9a1d-1f8c389127a7'),
    ('education', 'e40fa1c6-13a1-4a02-91cd-0eaade11864d'),
    ('owns_net', 'bd4802c6-7bdb-40f0-aac1-18cc6df7da6e'),
    ('has_ac', 'e279587c-975f-433a-adab-1ad563876af6'),
    ('net_lastnight', 'b5b1ff58-c8df-4890-9b1c-0cd40ce6edc0'),
    ('home', '4d0ae478-3893-4b46-ab39-d6848c69245d')]


In [576]:
from responses import get_response_df

def ref_translation(eng_form, other_form):
    vals = [(a['ref'], b['ref']) for a, b 
            in zip(eng_form['fields'], other_form['fields'])]
    lookup = dict(vals)
    rev_lookup = {v:k for k,v in lookup.items()}
    return lookup, rev_lookup


def get_filtered_responses(survey_user, eng_form, other_form, shortcodes, questions, db_cnf):
    ref_lookup, rev_ref_lookup = ref_translation(eng_form, other_form)
    refs = [ref_lookup[r] for _, r in questions]
    responses = get_response_df(survey_user, shortcodes, refs, db_cnf)

    _, last_ref = questions[-1]
    
    answered = responses[responses.question_ref == last_ref].userid.unique()    

    return responses[responses.userid.isin(answered)].reset_index(drop=True)


filtered_res = get_filtered_responses(survey_user, forms[0], forms[1], ['baselinehin', 'baselinehinexc'], questions, cnf['chatbase'])

In [ ]:
from clustering import add_res_cols
from forms import response_translator

def col_translators(a, b, questions):
    lookup, _ = ref_translation(a, b)

    refs = [ref for _, ref in questions]
    ts = { qt['ref']: response_translator(q, qt)
                   for qt, q in zip(a['fields'], b['fields'])                   if qt['ref'] in refs}

    return [(n, lookup[ref], ts[ref]) for n, ref in questions]

col_names = col_translators(forms[0], forms[1], questions)
rr = add_res_cols(col_names, shape_df(filtered_res)) \
    .reset_index(drop=True) \
    .dropna()

In [582]:
dd = rr.merge(districts, left_on='md:clusterid', right_on='disthash')

dd['under_net'] = dd.net_lastnight == 'Yes'
dd['malaria'] = dd.malaria_incidence == 'Yes'
dd['malaria_now'] = dd.malaria_now == 'Yes'
dd['kutcha'] = dd.home == 'Kutcha (made of mud, tin, straw)'

In [579]:
def malaria_prob(groupby, key='malaria'):
    d = groupby[key].value_counts(normalize=True)
    d.name = 'count'
    d = d.reset_index(level=-1)
    d = d[d[key] == True].drop(columns=key)
    return d

In [580]:
def stats(df):
    targets = df.kutcha.sum()
    tot = df.shape[0]
    non_target = tot - targets
    return pd.Series([targets/tot, targets, non_target, tot], index=['perc', 'target', 'non_target', 'tot'])

perc = dd.groupby('disthash').apply(stats).reset_index()

In [581]:
stratum = cnf['strata'][0]

stratum

{'name': 'kutchas',
 'targeting': {'age_min': 20},
 'per_cluster_pop': 100,
 'surveys': [{'shortcode': 'baselineeng',
   'cluster_question': {'name': 'district', 'ref': 'md:clusterid'},
   'target_questions': [{'name': 'home',
     'ref': '4d0ae478-3893-4b46-ab39-d6848c69245d',
     'op': 'equal',
     'value': 'A'}],
   'exclude_questions': [{'name': 'home',
     'ref': '4d0ae478-3893-4b46-ab39-d6848c69245d',
     'op': 'equal',
     'value': 'B'}]},
  {'shortcode': 'baselinehin',
   'cluster_question': {'name': 'district', 'ref': 'md:clusterid'},
   'target_questions': [{'name': 'home',
     'ref': '4d0ae478-3893-4b46-ab39-d6848c69245d',
     'op': 'equal',
     'value': 'A'}],
   'exclude_questions': [{'name': 'home',
     'ref': '4d0ae478-3893-4b46-ab39-d6848c69245d',
     'op': 'equal',
     'value': 'B'}]},
  {'shortcode': 'baselinehinexc',
   'cluster_question': {'name': 'district', 'ref': 'md:clusterid'},
   'target_questions': [{'name': 'home',
     'ref': '4d0ae478-3893-4b46-

In [553]:
from clustering import get_budget_lookup
from malaria import window, days_left, get_df

df = get_df(cnf)

stratum = cnf['strata'][0]
spend = m.get_spend(window())
budget_lookup = get_budget_lookup(df,
                                  stratum,
                                  float('inf'),
                                  days_left(cnf),
                                  window(),
                                  spend)

In [ ]:
bb = pd.Series(budget_lookup) / 100

with pd.option_context('display.max_rows', None):
    display(bb.sort_values())

In [560]:
bb.sort_values()[:80].sum() / 10

74685.96399999999

In [ ]:
from clustering import add_cluster, only_target_users

stratum = cnf['strata'][0]
surveys = stratum['surveys']
df = add_cluster(surveys, df)

In [292]:
targets = only_target_users(df, surveys, 'target_questions')

In [293]:
df['target'] = df.userid.isin(targets.userid)

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df.groupby('cluster').target.sum().sort_values())

In [355]:
def add_foo(r):
    r['budget'] = budget_lookup.get(r['disthash'], None)
    return r

perc = perc.apply(add_foo, 1)

In [481]:
too_expensive = [k for k, v in budget_lookup.items() if v > 10000]

In [515]:
sum([v for k, v in budget_lookup.items() if v > 10000])

91009

In [ ]:
# array(['786f66b7', '2e0a86c3', 'cbea7cbf', '4c6cbda0'], dtype=object)

In [563]:
cities[cities.include_audience == True].disthash.unique()

array(['dbaa941c', '188ae45b', '335580ac', '72d1d8e0', '8ce955f7',
       'c25e6958', 'd3ed46a4', '22a26988', '3c054e8f', 'ba353231',
       'a0530be6', 'e5b6ea7f', '15b5a749', '5c4dcb04', '9a766899',
       '3c73baaa', '07853f76', 'b406895f', 'd56fd7da', 'b923577c',
       'ce04df02', '196f4fd1', '5b788486', '0d1e3ccb', '0c4a9e7c',
       '59a2fd2c', '4d59f3e6', '9565590d', 'b27404c2', 'c12daf4e',
       'd9024a8b', '3aca460f', '8ed5f728', 'd86b84cb', 'e222a102',
       'e5f52cdf', '786f66b7', '42504ec5', '426d0912', '2e0a86c3',
       'c5c35762', '2ae8c08a', '749c3a9c', '9064b3f8', 'e765ab1c',
       '09c3966f', 'cfce48f2', '0b302692', 'e4f3449c', '247b0531',
       '419b24e9', 'c250de2d', 'e77e76a7', '5c8b8562', 'cbea7cbf',
       '5b1466b5', '5abbecc7', '3b34c260', '0c92f920', '4c6cbda0'],
      dtype=object)

In [ ]:
with pd.option_context('display.max_rows', None):
    display(perc.sort_values('tot'))

In [535]:
target_kutchas = perc[(perc.perc < .4) & (perc.tot > 40)].disthash

In [536]:
cities['include_audience'] = cities.disthash.isin(target_kutchas)

In [538]:
cities.to_csv('output/cities.csv', index=False)

In [ ]:
with pd.option_context('display.max_rows', None):
    display(dd.groupby('disthash').kutcha.sum().sort_values())

In [367]:
cities[cities.creative_group == 'exclusions'].disthash.unique()

array(['786f66b7', '2e0a86c3', 'cbea7cbf', '4c6cbda0'], dtype=object)

In [299]:
dd.home.value_counts()

Pucca (have cement/brick wall and floor    3270
Semi-pucca                                 1446
Kutcha (made of mud, tin, straw)           1194
Name: home, dtype: int64

In [493]:
high_malaria_dists = perc[(perc.perc > .25)].disthash.unique()
low_malaria_dists = perc[(perc.perc <= .15)].disthash.unique()
# low_malaria_dists = perc[(perc.perc == 0.0) & (perc.target >= 0)].disthash.unique()

# low_malaria_dists = cities[cities.include_audience == True].disthash.unique()

dd['dist_risk'] = 'med'
dd.loc[dd['md:clusterid'].isin(low_malaria_dists), 'dist_risk'] = 'low'
dd.loc[dd['md:clusterid'].isin(high_malaria_dists), 'dist_risk'] = 'high'

In [ ]:
dd.groupby('disthash').head(1)['dist_risk'].value_counts()

In [ ]:
malaria_prob(dd.groupby('home'))

In [ ]:
dd.groupby('education').has_ac.value_counts(1)

In [ ]:
malaria_prob(dd.groupby(['dist_risk', 'home']), 'malaria_now')

In [460]:
malaria_prob(dd.groupby(['home', 'has_ac', 'under_net']), 'malaria_now')

count
home                                    has_ac     under_net          
Kutcha (made of mud, tin, straw)        Don't know False      0.048387
                                                   True       0.038961
                                        No         False      0.036530
                                                   True       0.025039
                                        Yes        True       0.026316
Pucca (have cement/brick wall and floor Don't know False      0.015152
                                                   True       0.025974
                                        No         False      0.008246
                                                   True       0.009028
                                        Yes        False      0.009317
                                                   True       0.030769
Semi-pucca                              Don't know False      0.045455
                                                   True       0.047619
                                        No         False      0.016187
                                                   True       0.012210
                                        Yes        False      0.024390

In [497]:
malaria_prob(dd.groupby(['dist_risk', 'home']), 'malaria_now')

count
dist_risk home                                             
high      Kutcha (made of mud, tin, straw)         0.051392
          Pucca (have cement/brick wall and floor  0.017401
          Semi-pucca                               0.019093
low       Kutcha (made of mud, tin, straw)         0.019324
          Pucca (have cement/brick wall and floor  0.009715
          Semi-pucca                               0.011844
med       Kutcha (made of mud, tin, straw)         0.017812
          Pucca (have cement/brick wall and floor  0.006189
          Semi-pucca                               0.016822

In [48]:
# dd['include_audience'] = dd.dist_risk == 'low'

# cities = cities.merge(dd.groupby('disthash').head(1).reset_index(drop=True)[['disthash', 'include_audience']])

# cities.to_csv('output/cities.csv', index=False)

In [283]:
dd.groupby('dist_risk').malaria.value_counts()

dist_risk  malaria
high       False       921
           True        510
low        False      1927
           True        230
med        False      1736
           True        401
Name: malaria, dtype: int64

In [168]:
filter_time(dd, (2020,7,21), (2020,7,25)) \
    .pipe(lambda df: df[df.disthash.isin(low_malaria_dists)]).shape[0] / filter_time(dd, (2020,7,21), (2020,7,25)).shape[0]

0.48735408560311283

In [169]:
filter_time(dd, (2020,8,1), (2020,8,10)) \
    .pipe(lambda df: df[df.disthash.isin(low_malaria_dists)]).shape[0] / filter_time(dd, (2020,8,1), (2020,8,10)).shape[0]

0.26901062959934585

In [216]:
filter_time(dd[dd.disthash.isin(low_malaria_dists)], (2020,7,21), (2020,7,24)).home.value_counts(normalize=True)

Pucca (have cement/brick wall and floor    0.601583
Semi-pucca                                 0.262533
Kutcha (made of mud, tin, straw)           0.135884
Name: home, dtype: float64

In [215]:
filter_time(dd[dd.disthash.isin(low_malaria_dists)], (2020,8,1), (2020,8,7)).home.value_counts(normalize=True)

Pucca (have cement/brick wall and floor    0.494845
Semi-pucca                                 0.268041
Kutcha (made of mud, tin, straw)           0.237113
Name: home, dtype: float64

In [214]:
filter_time(dd[dd.disthash.isin(low_malaria_dists)], (2020,8,7), (2020,8,8)).home.value_counts(normalize=True)

Pucca (have cement/brick wall and floor    0.385965
Kutcha (made of mud, tin, straw)           0.315789
Semi-pucca                                 0.298246
Name: home, dtype: float64

In [174]:
saturated = perc[perc.non_target >= 125].disthash

In [175]:
saturated

22    2e0a86c3
34    4c6cbda0
52    786f66b7
81    cbea7cbf
Name: disthash, dtype: object

In [566]:
cities['creative_group'] = 'hindi'
# cities.loc[cities.disthash.isin(saturated), 'creative_group'] = 'exclusions'

In [567]:
cities

,id,distcode,distname,state,rad,lng,lat,disthash,creative_group,include_audience
0,245718296,363.0,Jamtara,Jharkhand,9.082666,86.802492,23.963216,94c3981f,hindi,False
1,6736171285,363.0,Jamtara,Jharkhand,4.793280,86.613861,24.045396,94c3981f,hindi,False
2,6736213486,363.0,Jamtara,Jharkhand,5.612525,87.161193,23.970470,94c3981f,hindi,False
3,566174725,369.0,Saraikela-Kharsawan,Jharkhand,2.375710,86.106653,22.815277,ba2a52b9,hindi,False
4,1323455437,369.0,Saraikela-Kharsawan,Jharkhand,11.198950,85.947944,22.791742,ba2a52b9,hindi,False
...,...,...,...,...,...,...,...,...,...,...
457,6591683648,175.0,Allahabad,Uttar Pradesh,7.658412,81.782379,25.450019,4c6cbda0,hindi,True
458,6591683649,175.0,Allahabad,Uttar Pradesh,10.965387,81.815529,25.457826,4c6cbda0,hindi,True
459,6591687870,175.0,Allahabad,Uttar Pradesh,16.329389,81.854603,25.529915,4c6cbda0,hindi,True
460,6591831677,175.0,Allahabad,Uttar Pradesh,18.265542,81.875911,25.483881,4c6cbda0,hindi,True


In [569]:
cities.to_csv('output/cities.csv', index=False)

In [502]:
# district fixed effects of puccaness

dd['pucca'] = dd.home != 'Kutcha (made of mud, tin, straw)'

malaria_prob(dd.groupby(['disthash', 'pucca'])).reset_index() \
    .groupby('disthash') \
    .filter(lambda df: df.shape[0] > 1) \
    .groupby('disthash') \
    .apply(lambda df: df.iloc[0]['count'] - df.iloc[1]['count']).quantile(0.5)

0.03352007469654528

In [227]:
malaria_prob(filter_time(dd, (2020,7,21), (2020,8,1)).groupby(['dist_risk', 'home']))

count
dist_risk home                                             
high      Kutcha (made of mud, tin, straw)         0.425703
          Pucca (have cement/brick wall and floor  0.322284
          Semi-pucca                               0.354839
low       Kutcha (made of mud, tin, straw)         0.136719
          Pucca (have cement/brick wall and floor  0.090982
          Semi-pucca                               0.135021
med       Kutcha (made of mud, tin, straw)         0.185965
          Pucca (have cement/brick wall and floor  0.188024
          Semi-pucca                               0.218919

In [252]:
malaria_prob(filter_time(dd, (2020,8,6), (2020,8,9)).groupby(['dist_risk', 'home']))

count
dist_risk home                                             
high      Kutcha (made of mud, tin, straw)         0.395522
          Pucca (have cement/brick wall and floor  0.297101
          Semi-pucca                               0.415842
low       Kutcha (made of mud, tin, straw)         0.125000
          Pucca (have cement/brick wall and floor  0.040541
          Semi-pucca                               0.155556
med       Kutcha (made of mud, tin, straw)         0.104294
          Pucca (have cement/brick wall and floor  0.163934
          Semi-pucca                               0.218750

In [246]:
malaria_prob(filter_time(dd, (2020,7,8), (2020,8,1)).groupby(['dist_risk']))

,count
dist_risk,
high,0.354808
low,0.107505
med,0.195302


In [250]:
malaria_prob(filter_time(dd, (2020,8,6), (2020,8,9)).groupby(['dist_risk']))

,count
dist_risk,
high,0.364611
low,0.094340
med,0.162420


In [ ]:
filter_time(dd, (2020,7,6), (2020,8,1)).malaria.value_counts(normalize=True)

In [238]:
filter_time(dd, (2020,7,6), (2020,8,1)).groupby('dist_risk').home.value_counts(normalize=True)

dist_risk  home                                   
high       Pucca (have cement/brick wall and floor    0.522115
           Kutcha (made of mud, tin, straw)           0.239423
           Semi-pucca                                 0.238462
low        Pucca (have cement/brick wall and floor    0.629817
           Semi-pucca                                 0.240365
           Kutcha (made of mud, tin, straw)           0.129817
med        Pucca (have cement/brick wall and floor    0.560403
           Semi-pucca                                 0.248322
           Kutcha (made of mud, tin, straw)           0.191275
Name: home, dtype: float64

In [241]:
filter_time(dd, (2020,8,7), (2020,8,9)).groupby('dist_risk').home.value_counts(normalize=True)

dist_risk  home                                   
high       Pucca (have cement/brick wall and floor    0.363914
           Kutcha (made of mud, tin, straw)           0.342508
           Semi-pucca                                 0.293578
low        Pucca (have cement/brick wall and floor    0.443182
           Kutcha (made of mud, tin, straw)           0.295455
           Semi-pucca                                 0.261364
med        Pucca (have cement/brick wall and floor    0.493827
           Semi-pucca                                 0.253968
           Kutcha (made of mud, tin, straw)           0.252205
Name: home, dtype: float64

In [243]:
filter_time(dd, (2020,7,6), (2020,8,1)).malaria.value_counts(normalize=True)

False    0.806308
True     0.193692
Name: malaria, dtype: float64

In [245]:
filter_time(dd, (2020,8,6), (2020,8,9)).malaria.value_counts(normalize=True)

False    0.781897
True     0.218103
Name: malaria, dtype: float64

In [198]:
filter_time(dd, (2020,7,6), (2020,8,1)).groupby('dist_risk').home.value_counts(normalize=True)

dist_risk  home                                   
high       Pucca (have cement/brick wall and floor    0.522115
           Kutcha (made of mud, tin, straw)           0.239423
           Semi-pucca                                 0.238462
low        Pucca (have cement/brick wall and floor    0.625341
           Semi-pucca                                 0.238667
           Kutcha (made of mud, tin, straw)           0.135991
med        Pucca (have cement/brick wall and floor    0.571429
           Semi-pucca                                 0.249540
           Kutcha (made of mud, tin, straw)           0.179031
Name: home, dtype: float64

In [196]:
filter_time(dd, (2020,8,6), (2020,8,9)).groupby('dist_risk').home.value_counts(normalize=True)

dist_risk  home                                   
high       Pucca (have cement/brick wall and floor    0.369973
           Kutcha (made of mud, tin, straw)           0.359249
           Semi-pucca                                 0.270777
low        Pucca (have cement/brick wall and floor    0.475410
           Semi-pucca                                 0.268852
           Kutcha (made of mud, tin, straw)           0.255738
med        Pucca (have cement/brick wall and floor    0.485477
           Kutcha (made of mud, tin, straw)           0.259336
           Semi-pucca                                 0.255187
Name: home, dtype: float64

In [ ]:
filter_time(dd, (2020,8,1), (2020,9,1)).pipe(lambda df: df[df.dist_risk == 'low'])

In [399]:
malaria_prob(dd.groupby(['dist_risk', 'under_net']))

KeyError: 'dist_risk'

In [398]:
malaria_prob(dd.groupby(['under_net']))

,count
under_net,
False,0.190691
True,0.208074


In [403]:
malaria_prob(dd.groupby(['dist_risk', 'home']))

count
dist_risk home                                             
high      Kutcha (made of mud, tin, straw)         0.404711
          Pucca (have cement/brick wall and floor  0.308943
          Semi-pucca                               0.363636
low       Kutcha (made of mud, tin, straw)         0.129747
          Pucca (have cement/brick wall and floor  0.086731
          Semi-pucca                               0.140481
med       Kutcha (made of mud, tin, straw)         0.164969
          Pucca (have cement/brick wall and floor  0.171990
          Semi-pucca                               0.202055

In [ ]:
malaria_prob(dd.groupby(['owns_net']))

In [409]:
dd.owns_net.value_counts()

Yes           4597
No            1580
Don't know      71
Name: owns_net, dtype: int64

In [418]:
dd.groupby(['has_ac']).owns_net.value_counts()

has_ac      owns_net  
Don't know  Yes            265
            No              71
            Don't know      32
No          Yes           3903
            No            1288
            Don't know      31
Yes         Yes            429
            No             221
            Don't know       8
Name: owns_net, dtype: int64

In [431]:
dd.groupby(['dist_risk']).home.value_counts(1)

dist_risk  home                                   
high       Pucca (have cement/brick wall and floor    0.493127
           Kutcha (made of mud, tin, straw)           0.267468
           Semi-pucca                                 0.239404
low        Pucca (have cement/brick wall and floor    0.611514
           Semi-pucca                                 0.245240
           Kutcha (made of mud, tin, straw)           0.143246
med        Pucca (have cement/brick wall and floor    0.531794
           Semi-pucca                                 0.254355
           Kutcha (made of mud, tin, straw)           0.213850
Name: home, dtype: float64

In [430]:
dd.groupby(['dist_risk', 'malaria']).under_net.value_counts(1)

dist_risk  malaria  under_net
high       False    True         0.561896
                    False        0.438104
           True     True         0.584843
                    False        0.415157
low        False    True         0.508621
                    False        0.491379
           True     True         0.508547
                    False        0.491453
med        False    True         0.543190
                    False        0.456810
           True     True         0.555012
                    False        0.444988
Name: under_net, dtype: float64

In [405]:
malaria_prob(dd.groupby(['dist_risk', 'under_net']))

count
dist_risk under_net          
high      False      0.335553
          True       0.356784
low       False      0.106089
          True       0.106061
med       False      0.174330
          True       0.181310

In [428]:
dd.groupby(['home', 'has_ac']).under_net.value_counts(1)

home                                     has_ac      under_net
Kutcha (made of mud, tin, straw)         Don't know  True         0.553957
                                                     False        0.446043
                                         No          True         0.593315
                                                     False        0.406685
                                         Yes         True         0.655172
                                                     False        0.344828
Pucca (have cement/brick wall and floor  Don't know  True         0.538462
                                                     False        0.461538
                                         No          True         0.519481
                                                     False        0.480519
                                         Yes         False        0.622093
                                                     True         0.377907
Semi-pucca                           

In [426]:
dd.groupby(['home']).owns_net.value_counts(1)

home                                     owns_net  
Kutcha (made of mud, tin, straw)         Yes           0.737049
                                         No            0.247253
                                         Don't know    0.015699
Pucca (have cement/brick wall and floor  Yes           0.718449
                                         No            0.270184
                                         Don't know    0.011367
Semi-pucca                               Yes           0.773169
                                         No            0.219054
                                         Don't know    0.007777
Name: owns_net, dtype: float64

In [422]:
malaria_prob(dd.groupby(['home', 'dist_risk', 'under_net']))

count
home                                    dist_risk under_net          
Kutcha (made of mud, tin, straw)        high      False      0.369565
                                                  True       0.427562
                                        low       False      0.103704
                                                  True       0.149171
                                        med       False      0.154229
                                                  True       0.172414
Pucca (have cement/brick wall and floor high      False      0.300771
                                                  True       0.315678
                                        low       False      0.100407
                                                  True       0.070261
                                        med       False      0.158516
                                                  True       0.184713
Semi-pucca                              high      False      0.376404
                                                  True       0.354167
                                        low       False      0.127358
                                                  True       0.148936
                                        med       False      0.228000
                                                  True       0.182635

In [423]:
malaria_prob(dd.groupby(['state', 'dist_risk', 'under_net']))

count
state         dist_risk under_net          
Chhatisgarh   high      False      0.431373
                        True       0.283186
              low       False      0.100000
                        True       0.117647
              med       False      0.127273
                        True       0.192090
Jharkhand     high      False      0.425743
                        True       0.395639
              low       False      0.092593
                        True       0.124138
              med       False      0.169014
                        True       0.157143
Uttar Pradesh high      False      0.312187
                        True       0.349376
              low       False      0.107660
                        True       0.103125
              med       False      0.184406
                        True       0.182888

In [ ]:
X = dd[['education', 'home', 'dist_medical', 'has_ac', 'owns_net']]


for col in X.columns:
    X[col] = X[col].astype('category')
    X[col] = X[col].cat.codes

In [ ]:
X

In [423]:
X.shape

(4474, 5)

In [425]:
y = dd['malaria']

In [ ]:
y.shape

In [542]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()

model.fit(X[:2000], y[:2000])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:

from sklearn.tree import plot_tree

plot_tree(model)

In [300]:
dd[(dd.owns_net == 'Yes') & 
   (dd.has_ac == 'No') & 
   (dd.home == 'Kutcha (made of mud, tin, straw)')] \
   .malaria_incidence.value_counts(normalize=True)

No            0.636550
Yes           0.268994
Don’t know    0.094456
Name: malaria_incidence, dtype: float64

In [ ]:
dd

In [330]:
def times(df, mi, ma):
    return df[(df.timestamp > mi) & (df.timestamp < ma)]    


times(dd, datetime(2020,7,22,tzinfo=timezone.utc), datetime(2020,7,24,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.846106
Yes           0.113693
Don't know    0.040201
Name: has_ac, dtype: float64

In [331]:
times(dd, datetime(2020,7,24,tzinfo=timezone.utc), datetime(2020,7,25,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.797244
Yes           0.147638
Don't know    0.055118
Name: has_ac, dtype: float64

In [332]:
times(dd, datetime(2020,7,25,tzinfo=timezone.utc), datetime(2020,7,26,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.862589
Yes           0.090426
Don't know    0.046986
Name: has_ac, dtype: float64

In [328]:
times(dd, datetime(2020,7,26,tzinfo=timezone.utc), datetime(2020,7,28,tzinfo=timezone.utc)) \
    ['has_ac'] \
    .value_counts(normalize=True)

No            0.825040
Yes           0.126806
Don't know    0.048154
Name: has_ac, dtype: float64

In [ ]:
dd

In [301]:
dd.malaria_incidence.value_counts(normalize=True)

No            0.670988
Yes           0.191775
Don’t know    0.137237
Name: malaria_incidence, dtype: float64

In [ ]:
# responses should be converted and translated...